# Bachelor Thesis Project

### *How to create an NLP Model for predicting noun capilaization in German?*

In [1]:
import sys
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import *
import string
import os
from bs4 import BeautifulSoup
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from pathlib import Path

### 1 Open required files

Currently only used folder AA - next steps: algorithm for automating the process with the other folders

In [4]:
files = []        
# open all files
for filename in os.listdir(os.getcwd() + "\\extracted\\AA"):
    with open(os.path.join(os.getcwd() + "\\extracted\\AA", filename), 'rt', encoding="utf-8") as f:
        files.append(f.read())

In [3]:
#### DEBUGGING ####
#### only one file
cur_dir = os.getcwd()

data = open(cur_dir + "\\extracted\\AA\\wiki_00", "rt", encoding="utf-8")
data = data.read()
###################

In [5]:
# backup, so no need to run again open files
data = files
# encode in utf-8
data = [x.encode('utf-8').decode('utf-8') for x in data]

In [6]:
# removing double space, chars and line breaks
clean_data = [((x.replace("\xa0", " ")).replace("\n", " ")).replace("  ", " ") for x in data] 

In [7]:
# remove <doc> tag, link and article id in beginning
clean_data = [re.sub(r'<.+?> ', '', x) for x in clean_data]

In [8]:
# remove digits, and the following words f.e. 1. August 2020 - 1. and 2020 are removed
clean_data = [re.sub("\S*\d\S*", '', x).strip() for x in clean_data] #remover words after .

In [9]:
# tokenize in list of articles with sentences as list
token_data = [sent_tokenize(x) for x in clean_data]
token_data[0:1]

[['Alan Smithee Alan Smithee steht als Pseudonym für einen fiktiven Regisseur, der Filme verantwortet, bei denen der eigentliche Regisseur seinen Namen nicht mit dem Werk in Verbindung gebracht haben möchte.',
  'Von  bis  wurde es von der Directors Guild of America (DGA) für solche Situationen empfohlen, seither ist es Thomas Lee.',
  '"Alan Smithee" ist jedoch weiterhin in Gebrauch.',
  'Alternative Schreibweisen sind unter anderem die Ursprungsvariante "Allen Smithee" sowie "Alan Smythee" und "Adam Smithee".',
  'Auch zwei teilweise asiatisch anmutende Schreibweisen "Alan Smi Thee" und "Sumishii Aran" gehören – so die Internet Movie Database – dazu.',
  'Das Pseudonym entstand  infolge der Arbeiten am Western-Film "Death of a Gunfighter" (deutscher Titel "Frank Patch – Deine Stunden sind gezählt").',
  'Regisseur Robert Totten und Hauptdarsteller Richard Widmark gerieten in einen Streit, woraufhin Don Siegel als neuer Regisseur eingesetzt wurde.',
  'Der Film trug nach Abschluss der

In [10]:
# flatten list = one big list with multiple sentences - no article recognition anymore - usefull (?)
token_data = [item for sublist in token_data for item in sublist]

In [11]:
# remove punctuation
token_data = [re.sub(r'[^\w\s]', '', x).strip() for x in token_data]

In [12]:
# tokenize all words in sentences = on bis list, with list of articles with tokenized words
token_data = [word_tokenize(x) for x in token_data]
token_data

[['Alan',
  'Smithee',
  'Alan',
  'Smithee',
  'steht',
  'als',
  'Pseudonym',
  'für',
  'einen',
  'fiktiven',
  'Regisseur',
  'der',
  'Filme',
  'verantwortet',
  'bei',
  'denen',
  'der',
  'eigentliche',
  'Regisseur',
  'seinen',
  'Namen',
  'nicht',
  'mit',
  'dem',
  'Werk',
  'in',
  'Verbindung',
  'gebracht',
  'haben',
  'möchte'],
 ['Von',
  'bis',
  'wurde',
  'es',
  'von',
  'der',
  'Directors',
  'Guild',
  'of',
  'America',
  'DGA',
  'für',
  'solche',
  'Situationen',
  'empfohlen',
  'seither',
  'ist',
  'es',
  'Thomas',
  'Lee'],
 ['Alan', 'Smithee', 'ist', 'jedoch', 'weiterhin', 'in', 'Gebrauch'],
 ['Alternative',
  'Schreibweisen',
  'sind',
  'unter',
  'anderem',
  'die',
  'Ursprungsvariante',
  'Allen',
  'Smithee',
  'sowie',
  'Alan',
  'Smythee',
  'und',
  'Adam',
  'Smithee'],
 ['Auch',
  'zwei',
  'teilweise',
  'asiatisch',
  'anmutende',
  'Schreibweisen',
  'Alan',
  'Smi',
  'Thee',
  'und',
  'Sumishii',
  'Aran',
  'gehören',
  'so',
 

In [13]:
# stopwords
de_stop_words = stopwords.words('german')
de_stop_words_cap = [x.capitalize() for x in de_stop_words]
de_stop_words_dict = {de_stop_words_cap[i]: de_stop_words[i] for i in range(len(de_stop_words_cap))} 

In [14]:
#replace stop words with lower case (in case at the beginning of a sentence)
def replace_matched_items(word_list, dictionary):
    for lst in word_list:
        for ind, item in enumerate(lst):
            lst[ind] = dictionary.get(item, item)

In [15]:
replace_matched_items(token_data, de_stop_words_dict)

In [16]:
model = Word2Vec(token_data)

In [17]:
words = list(model.wv.vocab)
#print(words)

### Method implementation

In [201]:
sim_low = (model.wv.most_similar("neuen", topn = len(words)))
for i in sim_low:
    #print(i[0])
    if "film" == i[0]:
        print("hi")
        word_low = i[0]
        val_low = i[1]
        
    if "Film" == i[0]:
        word_cap = i[0]
        val_cap = i[1]
print(val_low, val_cap)
if val_low > val_cap:
    relative_val = val_low/val_cap
else:
    relative_val = val_cap/val_low

count_low = model.wv.vocab["film"].count
count_cap = model.wv.vocab["Film"].count
print(count_low, count_cap)
if count_low > count_cap:
    relative_count = count_low/count_cap
else:
    relative_count = count_cap/count_low

    
if relative_count > relative_val:
    if count_low > count_cap:
        # word = low
    else:
        # word = cap
else:
    if val_low > val_cap:
        # val low
    else:
        # val cap

SyntaxError: unexpected EOF while parsing (<ipython-input-201-d7f4c2430252>, line 28)

In [ ]:
if "den" == 

In [210]:
input_x = "der regisseur filmt einen neuen film am set"

In [144]:
"essen" in words

True

In [211]:
split = input_x.split()
split_clean_orig = [x for x in split if x]# not in de_stop_words]
split_clean_low = [x.lower() for x in split]# if x not in de_stop_words]
split_clean_cap = [x.capitalize() for x in split]# if x not in de_stop_words]
split_clean_cap


split_clean_orig

['der', 'regisseur', 'filmt', 'einen', 'neuen', 'film', 'am', 'set']

In [ ]:
###### TESTING

In [215]:
## this should work

l = 0
found = [] # list of words found in vocab
unknown = [] # list of words not found in vocab
output_x = []

while l < (len(split_clean_cap)): 
  #  try:
        for i in split:
            if split_clean_low[l] in words:
                if split_clean_cap[l] in words:
                
                    if l > 0:
                        sim_cap = (model.wv.most_similar(found[-1], topn = len(words)))
                        if found[-1].istitle():            
                            found.append(split_clean_low[l])
                            output_x.append(split_clean_low[l])
                            print("0")
                            l += 1
            
                        else:
                            for k in sim_cap:
                                if split_clean_low[l] == k[0]:
                                    word_low = i[0]
                                    val_low = i[1]
                                else:
                                    val_low = -1
                            for k in sim_cap:
                                if split_clean_cap[l] == k[0]:
                                    word_cap = k[0]
                                    val_cap = k[1]
                                else:
                                    val_cap = -1

                            if val_low > val_cap:
                                relative_val = val_low/val_cap
                            else:
                                relative_val = val_cap/val_low

                            count_low = model.wv.vocab[split_clean_low[l]].count
                            count_cap = model.wv.vocab[split_clean_cap[l]].count

                            if count_low > count_cap:
                                relative_count = count_low/count_cap
                            else:
                                relative_count = count_cap/count_low


                            if relative_count > relative_val:
                                if count_low > count_cap:
                                    found.append(split_clean_low[l])
                                    output_x.append(split_clean_low[l])
                                    print("1")
                                    l += 1
                                else:
                                    found.append(split_clean_cap[l])
                                    output_x.append(split_clean_cap[l])
                                    print("2")
                                    l += 1
                            else:
                                if val_low > val_cap:
                                    found.append(split_clean_low[l])
                                    output_x.append(split_clean_low[l])
                                    print("3")
                                    l += 1
                                else:
                                    found.append(split_clean_cap[l])
                                    output_x.append(split_clean_cap[l])
                                    print("4")
                                    l += 1

                
                    else:
                        count_low = model.wv.vocab[split_clean_low[l]].count
                        count_cap = model.wv.vocab[split_clean_cap[l]].count
                        if count_low > count_cap:
                            found.append(split_clean_low[l])
                            output_x.append(split_clean_low[l])
                            print("5")
                            l += 1
                        elif count_low < count_cap:
                            found.append(split_clean_cap[l])
                            output_x.append(split_clean_cap[l])
                            print("6")
                            l += 1
                
                else:
                    found.append(split_clean_low[l])
                    output_x.append(split_clean_low[l])
                    print("7")
                    l += 1
                
            elif split_clean_cap[l] in words:
                found.append(split_clean_cap[l])
                output_x.append(split_clean_cap[l])
                print("8")
                l += 1
            else:
                unknown.append(split_clean_cap[l].upper())
                output_x.append(split_clean_cap[l].upper())
                print("unknown")
                l += 1
#     except:
#         print("Oopsi")

7
8
unknown
7
1
2
7
2


In [213]:
found

['der', 'Regisseur', 'einen', 'neuen', 'Film', 'am', 'Set']

In [216]:
output_x

['der', 'Regisseur', 'FILMT', 'einen', 'neuen', 'Film', 'am', 'Set']

In [185]:
for i in found:
    if i.lower() in split:
        input_x = input_x.replace(i.lower(), i)
        
        
for j in unknown:
    for c in split:
        if j.lower() == c.lower():
            test = [c.replace(c, j.upper()) for x in split if x == c.lower()]
  # for c in split]
   #        output2 = c.replace(j.lower(), j.upper())

input_x

'der Regisseur Filmt einen neuen Film'

In [ ]:
string.replace(old, new, count)

if 

In [ ]:
new_split = [x]

In [ ]:
found_word_dict = {de_stop_words_cap[i]: de_stop_words[i] for i in range(len(de_stop_words_cap))} 

In [579]:
l = 0
cap_or_uncap = []

for found[l] in found:
    sim_found = model.wv.most_similar(found[l], topn = len(words))
    for i in sim_found:
        if l < (len(found)-1):
            if found[l+1] == i[0]:
                print(i[0])
                cap_or_uncap.append(i[0])
                l += 1
                

sim_found = model.wv.most_similar(found[1], topn = len(words))
for i in sim_found:
    if found[0] == i[0]:
        cap_or_uncap.append(i[0])
        print(i[0])
        



neuen
Film
Regisseur


In [553]:
test = model.wv.most_similar("film", topn = len(words))
for i in test:
    if i[1] > 0 and i[1] < 1:
        if (i[0]).lower() == "regisseur":
            print(i)


('Regisseur', 0.40539222955703735)


In [549]:
for i[1] in result:
    print(i[1][0])

('Produzent', 0.9040467739105225)
('militärischen', 0.6416114568710327)
[[...], [...], ('Hauptdarsteller', 0.7578281760215759), ('Oscar', 0.748108983039856), ('Hauptrolle', 0.7176164388656616), ('Madonna', 0.7163093090057373), ('Brando', 0.7131593823432922), ('Hitchcock', 0.696880578994751), ('Eastwood', 0.694446861743927), ('Western', 0.6872900724411011), ('Roman', 0.6742212176322937), ('Song', 0.6677567362785339), ('Dean', 0.6672717332839966), ('Filme', 0.6643990278244019), ('Filmen', 0.6623098850250244), ('Band', 0.6622206568717957), ('Spielfilm', 0.6584795713424683), ('Komödie', 0.6576427817344666), ('Drehbuch', 0.656978964805603), ('Nicholson', 0.6564452648162842), ('Chaplin', 0.6515282392501831), ('Autor', 0.6503322124481201), ('Album', 0.649824857711792), ('Star', 0.6480388641357422), ('DiCaprio', 0.6469562649726868), ('Produzent', 0.6466948390007019), ('Marlon', 0.6454980969429016), ('Auftritt', 0.643793523311615), ('Leone', 0.642557680606842), ('Kinofilm', 0.6410809755325317),

In [194]:
model.wv.most_similar("neu", topn = len(words))

[('neue', 0.5886570811271667),
 ('Europakarte', 0.5473616719245911),
 ('worden', 0.5339658856391907),
 ('umstrukturiert', 0.5300506353378296),
 ('aufgelöst', 0.5272859930992126),
 ('liberalisiert', 0.5262049436569214),
 ('mehrmals', 0.49914026260375977),
 ('modernisiert', 0.4938057065010071),
 ('Zubau', 0.492923766374588),
 ('Landkreise', 0.49270695447921753),
 ('endgültig', 0.4908864498138428),
 ('neuen', 0.4885815382003784),
 ('eingeführt', 0.48614490032196045),
 ('kreisangehörigen', 0.4838840067386627),
 ('offiziell', 0.48142990469932556),
 ('Gebäude', 0.4811995029449463),
 ('Synagoge', 0.4811576306819916),
 ('Klosterschule', 0.4808371365070343),
 ('zusammengelegt', 0.48066210746765137),
 ('abgerissen', 0.47976845502853394),
 ('Häuser', 0.47788962721824646),
 ('wurde', 0.4758920669555664),
 ('abgeschafft', 0.4755474030971527),
 ('Städteordnung', 0.4753915071487427),
 ('erneuert', 0.47360703349113464),
 ('seitdem', 0.472442626953125),
 ('komplett', 0.47130072116851807),
 ('Cham', 0.4

In [164]:
print(model.predict_output_word(["es", "wird", "verwendet"]))

[('Häufig', 0.006112288), ('dazu', 0.0048097847), ('Hierbei', 0.0047626398), ('Manchmal', 0.003560061), ('Heute', 0.0034889164), ('Dabei', 0.0034139317), ('Adjektiv', 0.0033082785), ('verwendet', 0.0028227302), ('häufig', 0.0027368774), ('Meist', 0.0025671604)]


### Stopped here

### --------------------------------------------------------------- TESTING AREA ----------------------------------------------------------------- ###

In [ ]:
#uncapitalize = lambda x: x[:1].lower() + x[1:] if x else '' ###HOLY uncapilaize two following cpaital words

final_clean = list(map(uncapitalize, list)); 

In [ ]:
final_clean

In [ ]:
final_clean2 = [word_tokenize(x) for x in final_clean]

In [ ]:
final_clean2

In [ ]:
model = Word2Vec(flat_list)

words = print(list(model.wv.vocab))

In [ ]:
print(model["das"])

model = model.save('model.bin')

In [ ]:
result = model.most_similar(positive=['Regisseur'])
print(result)

In [ ]:
######################################################

In [ ]:
model = gensim.models.Word2Vec.load('model.bin')

In [ ]:
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
F = plt.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
plt.xlim(-0.1, -1)
plt.ylim(0.1, 1)
plt.show()
#F(figsize=(10, 5)

In [ ]:
#clean_data = [re.sub(r'\w*\d\w*', '', x).strip() for x in clean_data] #remover words after number not dot